In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import ChatPromptTemplate
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import is_datetime64_any_dtype, is_numeric_dtype, is_object_dtype
import os
from pydantic import BaseModel
import json
import numpy as np

# LLM

## Preprocessing
- Return JSON's Headers and Description of Data

### Return JSON's Headers and Description of Data

- Tells LLM what files are available and what headers are in each file

In [2]:
def extract_headers(file_path):
    """
    Extract headers from a JSON file.

    Parameters:
        file_path (str): The path to the JSON file.

    Returns:
        list: List of headers from the JSON file.
    """
    with open(file_path, "r") as f:
        data = json.load(f)

    headers = []

    if isinstance(data, dict):
        # For a dictionary, get the keys
        headers = list(data.keys())
    elif isinstance(data, list) and data:
        first_item = data[0]
        if isinstance(first_item, dict):
            # For a list of dictionaries, get the keys from the first item
            headers = list(first_item.keys())

    return headers


def recursive_json_schema_extractor(directory):
    """
    Recursively walks through the given directory and extracts headers
    from all JSON files found.

    Parameters:
        directory (str): The root directory to start the recursive search.

    Returns:
        dict: A dictionary mapping JSON file paths to their header lists
    """
    schemas = {}
    for entry in os.listdir(directory):
        full_path = os.path.join(directory, entry)
        if os.path.isdir(full_path):
            # Recursively process subdirectories
            schemas.update(recursive_json_schema_extractor(full_path))
        elif entry.lower().endswith(".json"):
            try:
                headers = extract_headers(full_path)
                schemas[full_path] = headers
            except Exception as e:
                print(f"Error processing file {full_path}: {e}")
    return schemas


# Extract headers from all JSON files
data_dir = "./Data"
json_schemas = recursive_json_schema_extractor(data_dir)

# Print the results
for file_path, headers in json_schemas.items():
    print(f"\nFile: {file_path}")
    print("Headers:")
    for header in headers:
        print(f"  {header}")
    print("-" * 40)


File: ./Data/Adjusted_Ad_Campaign_Performance_Data.json
Headers:
  Time
  Campaign_ID
  Age_group
  Channel_Name
  Spending
  Number_of_Views
  Number_of_Leads
----------------------------------------

File: ./Data/Banking_KPI_Data.json
Headers:
  Time
  Number_of_New_Accounts
  Total_Base_Mn
----------------------------------------


## Description Generator

In [3]:
# Format json_schemas into a readable string for the prompt (without type info)
def format_schemas_for_prompt(schemas):
    formatted_str = ""
    for file_path, headers in schemas.items():
        file_name = os.path.basename(file_path)  # Get just the filename without path
        # Assume headers is now a list; join just the header names.
        headers_str = ", ".join(headers)
        formatted_str += f"{file_name}: [{headers_str}]\n"
    return formatted_str


# Pydantic model for column selection
class ColumnSelections(BaseModel):
    selections: list[tuple[str, str]]  # List of (filename, column) pairs


# Template for column selection
template = """Given the following JSON file headers and their data types, determine which columns would be relevant to answer the query.

Available JSON files and their headers (with types):
{json_headers}

Query: {query}

Respond only with a list of [filename, column] pairs, one per line, in this exact format:
[file1.json, column1]
[file1.json, column2]
[file2.json, column3]
...etc

Each pair should be unique and relevant to the query."""

prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="llama3.2")


# Function to parse LLM output into ColumnSelections
def parse_llm_response(response: str) -> ColumnSelections:
    lines = response.strip().split("\n")
    selections = []
    seen_pairs = set()  # To ensure uniqueness

    for line in lines:
        # Remove brackets and split by comma
        clean_line = line.strip("[]").split(",")
        if len(clean_line) == 2:
            filename = clean_line[0].strip()
            column = clean_line[1].strip()
            pair = (filename, column)

            # Only add if we haven't seen this combination before
            if pair not in seen_pairs:
                selections.append(pair)
                seen_pairs.add(pair)

    return ColumnSelections(selections=selections)


# Create the chain with structured output
chain = prompt | model | parse_llm_response

# Example usage with json_schemas:
formatted_headers = format_schemas_for_prompt(json_schemas)
result = chain.invoke(
    {"json_headers": formatted_headers, "query": "Show me the revenue trends"}
)

# Print results
print("\nRelevant columns:")
for filename, column in result.selections:
    print(f"- {filename}: {column}")


Relevant columns:
- Adjusted_Ad_Campaign_Performance_Data.json: Spending
- Adjusted_Ad_Campaign_Performance_Data.json: Number_of_Views


In [4]:
import json
import numpy as np


class NumpyEncoder(json.JSONEncoder):
    """
    Custom JSON Encoder that converts numpy data types
    into native Python types so they can be serialized.
    """

    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)


def collect_statistics_report(result: ColumnSelections) -> list:
    """
    Collects comprehensive statistical report data for selected columns,
    returning a JSON-serializable list of statistics per (filename, column) pair.

    Parameters:
        result (ColumnSelections): The Pydantic model containing the list of (filename, column) pairs.

    Returns:
        list: A list of dictionaries with detailed statistics for each selected column.
    """
    report = []
    for filename, column in result.selections:
        try:
            # Read the JSON file
            df = pd.read_json(f"./Data/{filename}")

            if pd.api.types.is_numeric_dtype(df[column]):
                stats = {
                    "Count": df[column].count(),
                    "Missing Values": df[column].isnull().sum(),
                    "Mean": df[column].mean(),
                    "Median": df[column].median(),
                    "Mode": (
                        df[column].mode().iloc[0]
                        if not df[column].mode().empty
                        else None
                    ),
                    "Std Dev": df[column].std(),
                    "Min": df[column].min(),
                    "Max": df[column].max(),
                    "Q1 (25th percentile)": df[column].quantile(0.25),
                    "Q3 (75th percentile)": df[column].quantile(0.75),
                    "IQR": df[column].quantile(0.75) - df[column].quantile(0.25),
                }
            else:
                # For categorical columns, calculate unique count, missing values, and frequency counts.
                value_counts = df[column].value_counts().to_dict()
                stats = {
                    "Unique Values": df[column].nunique(),
                    "Missing Values": df[column].isnull().sum(),
                    "Value Frequencies": value_counts,
                }
        except Exception as e:
            stats = {"error": str(e)}

        report.append({"filename": filename, "column": column, "statistics": stats})
    return report


# Example usage: Store the statistics in a JSON-serializable data structure.
stats_report = collect_statistics_report(result)

# Convert the report to a JSON string using the custom NumpyEncoder.
stats_report_json = json.dumps(stats_report, indent=4, cls=NumpyEncoder)
print(stats_report_json)

[
    {
        "filename": "Adjusted_Ad_Campaign_Performance_Data.json",
        "column": "Spending",
        "statistics": {
            "Count": 14068,
            "Missing Values": 0,
            "Mean": 0.40944839351720214,
            "Median": 0.23,
            "Mode": 0.02,
            "Std Dev": 0.5067380156190855,
            "Min": 0.0,
            "Max": 6.45,
            "Q1 (25th percentile)": 0.08,
            "Q3 (75th percentile)": 0.56,
            "IQR": 0.48000000000000004
        }
    },
    {
        "filename": "Adjusted_Ad_Campaign_Performance_Data.json",
        "column": "Number_of_Views",
        "statistics": {
            "Count": 14068,
            "Missing Values": 0,
            "Mean": 200.9016917827694,
            "Median": 109.0,
            "Mode": 1,
            "Std Dev": 270.0523566422892,
            "Min": 0,
            "Max": 6472,
            "Q1 (25th percentile)": 36.0,
            "Q3 (75th percentile)": 271.0,
            "IQR": 235.0


In [5]:
template = """Based on the following JSON statistics report for various columns, generate a comprehensive written summary of the findings.

JSON Statistics Report:
{stats_report}

Please provide a detailed summary including key insights, trends, and any anomalies in the data.
"""

prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="llama3.2")

chain = prompt | model
final_report = chain.invoke({"stats_report": stats_report_json})

print(final_report)

**Comprehensive Summary of the JSON Statistics Report**

The provided JSON statistics report contains information on two columns: "Spending" and "Number of Views". Both columns have identical statistical characteristics, such as counts, missing values, means, medians, modes, standard deviations, minimums, maximums, quartiles, and interquartile ranges (IQR). This suggests that the data is consistent across both columns.

**Key Insights:**

1. **Consistency across columns**: The identical statistical characteristics of both columns indicate a high degree of consistency in the data.
2. **Large number of observations**: Both columns have 14,068 observations, which is a significant sample size and should provide reliable insights into the underlying phenomena.
3. **Presence of missing values**: Neither column has any missing values, which indicates that the data is complete and free from errors.

**Trends:**

1. **Skewed distribution**: The "Spending" column follows a skewed distribution wi